## Trabajo Práctico 3: Detectando atributos de personas

### Grupo 4  

A partir de un conjunto de datos que contiene imagenes recortadas de personas, desarrollaremos un modelo que sea capaz de detectar si la persona tiene o no barba. Utilizaremos la columna "No_beard" como target. Los valores de salida pueden ser: "Sin Barba" y "Con Barba". 

### Setup inicial

#### Libs

In [1]:
import os

# Para especificar rutas de archivos y directorios
from pathlib import Path

# Lib para trabajar con arrays
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

import seaborn as sns

# Lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de 
# imágenes


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16
from IPython.display import HTML, display

# Libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,balanced_accuracy_score

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

#### Datasets

In [2]:
#Path con la ruta con las imágenes
IMAGES_DIR = Path('./dataset/img_align_celeba/img_align_celeba/')
ATRIBUTOS_DIR = Path('./dataset/list_attr_celeba.csv')
PARTICIONES_DIR = Path('./dataset/list_eval_partition.csv')

In [3]:
#Variables a utilizar
SIZE = 70
IMAGES_CHANNELS = 3
BATCH_SIZE = 50

In [4]:
#Utilizamos solamente la columna "No_Beard"
data = pd.read_csv(ATRIBUTOS_DIR,sep=',',usecols=['image_id','No_Beard'])

In [5]:
#Reemplazamos los valores = 1 por 'Si' y los valores = -1 por 'No'
data['No_Beard']=data['No_Beard'].replace([1], 'No_Barba')
data['No_Beard']=data['No_Beard'].replace([-1], 'Barba')

In [6]:
#Datos divididos en diferentes conjuntos para armar: train, test y validation
data_partition = pd.read_csv(PARTICIONES_DIR,sep=',',usecols=['image_id','partition'])

In [7]:
#Unifico data_partition con data para dividir el dataset en conjuntos: train, test y validation
df_merge = pd.merge(data, data_partition, how='inner', on = 'image_id')

In [8]:
#Chequeo que se unifiquen todos
df_merge.partition.isnull().sum()

In [9]:
#Conjunto Train, Test, Validation

train = df_merge[df_merge['partition'] == 0]
test = df_merge[df_merge['partition'] == 1]
validation = df_merge[df_merge['partition'] == 2]

#Eliminamos la columna del nro de particion
train = train.drop(['partition'],axis = 1)
test = test.drop(['partition'],axis = 1)
validation = validation.drop(['partition'],axis = 1)

### 1. Análisis exploratorio sobre el conjunto de datos

In [10]:
# Función para devolver ejemplos del dataset
def sample_images(dataset, n, figsize=(10, 5), image_width=SIZE, image_height=SIZE):
    samples = data.sample(n)
    fig, axes = plt.subplots(1, n, figsize=figsize)

    datagen = ImageDataGenerator(rescale=1/255.0)

    for i in range(n):
        image = load_img(os.path.join(dataset, samples.iloc[i]['image_id']))
        image = image.resize((image_width, image_height))
        image = img_to_array(image)
        image = datagen.standardize(image)

        axes[i].imshow(image)
        axes[i].axis('off')
        axes[i].set_title(f"{samples.iloc[i]['No_Beard']}")

    plt.tight_layout()
    plt.show()


#### Volumetría de los datos

In [11]:
print("Cantidad total de imágenes:", len(df_merge))


#### Distribución de la variable a predecir

In [12]:
vals_df = df_merge.No_Beard.value_counts()
names_df = vals_df.index
fig = plt.figure(figsize=(15, 5))

ax_train = fig.add_subplot(1, 3, 1)
ax_train.pie(vals_df, labels=names_df, autopct='%1.1f%%')
ax_train.set_title('Distribución conjunto total')

Podemos ver en los gráficos que la variable target se encuentra desbalanceada. Más del 80% de las imágenes de las personas en el dataset, no tienen barba. 

#### Balanceo del dataset

Realizamos undersampling para eliminar aleatoriamente muestras de la clase mayoritaria y así poder igualar el número de muestras de la clase minoritaria.

In [13]:
target_count = df_merge["No_Beard"].value_counts()
higher_category = list(target_count.index)[0]
np.random.seed(42)
indices = df_merge[df_merge["No_Beard"] == higher_category].index
sample_size = target_count[0] - target_count[1]

drop_sample = np.random.choice(indices, sample_size, replace = False)
df_merge_opt = df_merge.drop(drop_sample, axis = "index")

In [14]:
vals_df = df_merge_opt.No_Beard.value_counts()
names_df = vals_df.index
fig = plt.figure(figsize=(15, 5))

ax_train = fig.add_subplot(1, 3, 1)
ax_train.pie(vals_df, labels=names_df, autopct='%1.1f%%')
ax_train.set_title('Distribución conjunto total')

In [15]:
train = df_merge_opt[df_merge_opt['partition'] == 0]
test = df_merge_opt[df_merge_opt['partition'] == 1]
validation = df_merge_opt[df_merge_opt['partition'] == 2]

In [16]:
vals_train = train.No_Beard.value_counts()
names_train = vals_train.index
fig = plt.figure(figsize=(15, 5))

ax_train = fig.add_subplot(1, 3, 1)
ax_train.pie(vals_train, labels=names_train, autopct='%1.1f%%')
ax_train.set_title('Distribución conjunto Train')


vals_test = test.No_Beard.value_counts()
names_test = vals_test.index
ax_test = fig.add_subplot(1, 3, 2)
ax_test.pie(vals_test, labels=names_test, autopct='%1.1f%%')
ax_test.set_title('Distribución conjunto Test')

vals_val = validation.No_Beard.value_counts()
names_val = vals_val.index
ax_val = fig.add_subplot(1, 3, 3)
ax_val.pie(vals_val, labels=names_val, autopct='%1.1f%%')
ax_val.set_title('Distribución conjunto Validation')

plt.subplots_adjust(wspace=0.4)

plt.show()

In [17]:
print("Cantidad de imágenes y dimensiones en Train:", train.shape)

In [18]:
print("Cantidad de imágenes y dimensiones en Test:", test.shape)

In [19]:
print("Cantidad de imágenes y dimensiones en Validation:", validation.shape)

#### Estructura y tipo de las imágenes

El conjunto de imágenes originales tienen una dimensión de 178x218 pixeles y son a color, por lo tanto tienen 3 canales de profundidad. 

##### Ejemplos de las imágenes originales

In [20]:
sample_images(IMAGES_DIR, 5, figsize=(10, 5), image_width=178, image_height=218)

##### Ejemplos de las imágenes modificadas en tamaño y reescaladas. 

No contamos con una GPU para entrenar los modelos, por lo tanto: 
- Reducimos el tamaño a 70x70 pixeles: de esta manera, cada modelo tiene que aprender una menor cantidad de pesos, entonces, los resultados se obtienen más rápido. 
- Reescalamos los valores de las imágenes entre 0 y 1. 

In [21]:
#Lector de imágenes que se encarga de reescalarlas. 
images_reader = ImageDataGenerator(
    rescale=1/255
)

#Definimos un generador de DataFrame para cada conjunto de Datos. 

train_generator = images_reader.flow_from_dataframe(
    #DataFrame que contiene el nombre de las imagenes y la variable target. 
    dataframe = train,
    #Path que contiene todas las imágenes
    directory=IMAGES_DIR,
    #Columna del DataFrame que contiene el nombre de las imágenes
    x_col='image_id',
    #Columna target del DataFrame. 
    y_col='No_Beard',
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=False,
    target_size=(SIZE, SIZE)
)

test_generator = images_reader.flow_from_dataframe(
    #DataFrame que contiene el nombre de las imagenes y la variable target. 
    dataframe = test,
    #Path que contiene todas las imágenes
    directory=IMAGES_DIR,
    #Columna del DataFrame que contiene el nombre de las imágenes
    x_col='image_id',
    #Columna target del DataFrame. 
    y_col='No_Beard',
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=False,
    target_size=(SIZE, SIZE)
)

validation_generator = images_reader.flow_from_dataframe(
    #DataFrame que contiene el nombre de las imagenes y la variable target. 
    dataframe = validation,
    #Path que contiene todas las imágenes
    directory=IMAGES_DIR,
    #Columna del DataFrame que contiene el nombre de las imágenes
    x_col='image_id',
    #Columna target del DataFrame. 
    y_col='No_Beard',
    class_mode="binary",
    batch_size=BATCH_SIZE,
    shuffle=False,
    target_size=(SIZE, SIZE)
)

In [22]:
sample_images(IMAGES_DIR, 5)

### 2. Machine Learning

In [23]:
### Función para graficar la curva de aprendizaje: 
def learning_curve(historial):
    plt.plot(historial.history['accuracy'], label='train')
    plt.plot(historial.history['val_accuracy'], label='test')
    plt.title('Accuracy durante el entrenamiento')
    plt.ylabel('Accuracy')
    plt.xlabel('Época')
    plt.legend(loc='upper left')
    plt.show()

Para las redes neuronales de tipo MLP utilizamos los siguientes parámetros en común: 
- Primer capa **Flatten** que recibe una imágen de 70x70x3. 
- Función de activación en la capa de salida: **Sigmoide**. 
- Se trata de una clasificación binaria, que indica si la imágen de la persona tiene barba (No_Beard = 1) o no (No_Beard = 0). Utilizamos una función de error de tipo **Binary_crossentropy** ya que son redes que tienen una sóla salida entre 0 y 1. 
- La métrica utilizada es "Accuracy" ya que la distribución de la variable target está balanceada. 
- Empezamos utilizando 10 épocas, luego fuimos variando según cada caso. 
- El resto de los hiperparámetros lo fuimos modificando para cada red neuronal propuesta. 

Red Neuronal 1:
- MLP
- Capas ocultas: 2 densas, con 20 neuronas y 15 neuronas. 
- Capa de salida: Densa con 1 neurona. 
- Dropout: no utilizado.
- Función de activación: "relu" para las primeras capas. 
- Épocas: 10. 
- Tamaño de Batch: 256


In [24]:
model_mlp_1 = Sequential([
    Flatten(input_shape=(SIZE,SIZE,IMAGES_CHANNELS)), 
    
    Dense(20, activation='relu'),
    Dense(15, activation='relu'),
    
    Dense(1, activation='sigmoid'),
])

model_mlp_1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_1.summary()

In [ ]:
history_mlp_1 = model_mlp_1.fit(
    train_generator,
    epochs=10,
    batch_size=128,
    validation_data=test_generator
)